In [1]:
import os
import sys
import time
import glob
import datetime
import sqlite3
import numpy as np
import ConfigParser
from nltk import ngrams
import hashlib
import numpy.matlib

$Step 1$
Read parameters from the config file

In [2]:
g65_configFileLocation = './config'
Config = ConfigParser.ConfigParser()
Config.read(g65_configFileLocation)
def ConfigSectionMap(section):
    dict1 = {}
    options = Config.options(section)
    for option in options:
        try:
            dict1[option] = Config.get(section, option)
            if dict1[option] == -1:
                DebugPrint("skip: %s" % option)
        except:
            print("exception on %s!" % option)
            dict1[option] = None
    return dict1

In [3]:
# set pathes
msd_subset_path = ConfigSectionMap("MainSection")['dataset_location']
msd_code_path = ConfigSectionMap("MainSection")['code_location']
msd_subset_data_path=os.path.join(msd_subset_path,'data')
msd_subset_addf_path=os.path.join(msd_subset_path,'AdditionalFiles')
assert os.path.isdir(msd_subset_path),'wrong path'
assert os.path.isdir(msd_code_path),'wrong path'
sys.path.append( os.path.join(msd_code_path,'PythonSrc') )

# g65_selctedFeatures stores all the features
g65_selectedFeatures =ConfigSectionMap("MainSection")['features'].split("<g65SEP>")
num_feature = len(g65_selectedFeatures)

# g65_n is number of songs to process
g65_n = int(ConfigSectionMap("MainSection")['number_of_songs_to_process'])

# g65_r is number of rows in each band
g65_r = int(ConfigSectionMap("MainSection")['number_of_rows_each_band'])

# g65_b is number of bands
g65_b = int(ConfigSectionMap("MainSection")['number_of_bands'])

# g65_epsilon is the tolerant difference
g65_epsilon = int(ConfigSectionMap("MainSection")['tolerance_diff'])

#signiture is the raw value of each song
signiture = np.zeros([num_feature,g65_n])

In [4]:
import hdf5_getters as GETTERS

In [5]:
# extract all query features into the signiture matrix
s_counter = 0
for root, dirs, files in os.walk(msd_subset_data_path):
    files = glob.glob(os.path.join(root,'*'+'.h5'))
    for f in files :
        if s_counter == g65_n:
            break
        else:
            h5 = GETTERS.open_h5_file_read(f)
            f_counter = 0
            for features in g65_selectedFeatures:
                temp = str(getattr(GETTERS, 'get_'+features)(h5))
                signiture[f_counter,s_counter] = temp
                f_counter += 1
            s_counter += 1
            h5.close()
#print signiture

[[  1.77501590e+02   2.56730980e+02   3.40793020e+02 ...,   2.59839550e+02
    2.24574240e+02   1.64544850e+02]
 [  0.00000000e+00   0.00000000e+00   2.87000000e-01 ...,   0.00000000e+00
    4.76600000e+00   0.00000000e+00]
 [  0.00000000e+00   1.10000000e+01   6.00000000e+00 ...,   1.10000000e+01
    0.00000000e+00   4.00000000e+00]
 ..., 
 [  1.74237000e+02   2.45412000e+02   3.24795000e+02 ...,   2.52865000e+02
    2.20665000e+02   1.59643000e+02]
 [  1.26202000e+02   1.20103000e+02   1.10058000e+02 ...,   1.19992000e+02
    1.39487000e+02   1.07288000e+02]
 [  4.00000000e+00   4.00000000e+00   1.00000000e+00 ...,   4.00000000e+00
    4.00000000e+00   3.00000000e+00]]


$Step2$
Construct a new signiture matrix M for LSH with cosin distance

In [6]:
vec = np.random.randn(g65_b*g65_r,num_feature)
vec2 = vec*vec
vecNorm = np.sqrt(vec2.sum(axis=1))
vecNorm = np.matlib.repmat(vecNorm,num_feature,1)
hashV = vecNorm.T/vec
M = np.sign(hashV.dot(signiture))
#print M, M.shape


[[ 1.  1.  1. ...,  1.  1.  1.]
 [-1. -1. -1. ..., -1. -1. -1.]
 [-1. -1. -1. ..., -1. -1. -1.]
 ..., 
 [ 1.  1.  1. ...,  1.  1.  1.]
 [-1. -1. -1. ..., -1. -1. -1.]
 [-1. -1. -1. ..., -1. -1. -1.]] (60, 1000)
